In [ ]:
import torch
import numpy as np
from models import LIMUBertModel4Pretrain, ClassifierGRU
from config import PretrainModelConfig, ClassifierModelConfig, load_model_config, load_dataset_stats

# Asumimos que estamos en un notebook de Colab/Jupyter

# 1. Configuración y carga de modelos
def load_models(bert_path, classifier_path, dataset, version):
    # Cargar configuraciones
    bert_cfg = load_model_config('pretrain', 'base', 'v1', path_bert='config/limu_bert.json')
    classifier_cfg = load_model_config('classifier', 'gru', 'v1', path_classifier='config/classifier.json')
    
    # Imprimir la configuración del clasificador para verificar
    print("Classifier config:", classifier_cfg)
    
    # Inicializar modelos
    bert_model = LIMUBertModel4Pretrain(bert_cfg, output_embed=True)
    # Asumiendo que el modelo guardado tiene una capa lineal final de 6 a 10
    output_dim = 6  # o el valor correcto basado en tu modelo guardado
    classifier_model = ClassifierGRU(classifier_cfg, input=bert_cfg.hidden, output=output_dim)
    #classifier_model = ClassifierGRU(classifier_cfg, input=bert_cfg.hidden, output=classifier_cfg.linear_io[-1][1])
    
    # Imprimir la estructura del modelo para verificar
    print(classifier_model)
    
    # Cargar pesos pre-entrenados
    bert_model.load_state_dict(torch.load(bert_path, map_location='cpu'))
    
    # Cargar el estado del clasificador
    classifier_state = torch.load(classifier_path, map_location='cpu')
    
    # Imprimir las claves y formas del estado del clasificador
    for key, value in classifier_state.items():
        print(f"{key}: {value.shape}")
    
    # Intenta cargar el estado del clasificador
    classifier_model.load_state_dict(classifier_state, strict=False)
    
    return bert_model, classifier_model

# 2. Preprocesamiento de datos
def preprocess_data(raw_data, window_size=120, stride=60):
    windows = []
    for i in range(0, len(raw_data) - window_size + 1, stride):
        windows.append(raw_data[i:i+window_size])
    return np.array(windows)

# 3. Inferencia
def inference(bert_model, classifier_model, input_data):
    bert_model.eval()
    classifier_model.eval()
    
    with torch.no_grad():
        embeddings = bert_model(torch.FloatTensor(input_data))
        outputs = classifier_model(embeddings)
        
    return torch.softmax(outputs, dim=1).numpy()

# 4. Función principal modificada
def classify_imu_data(raw_data, seconds, sample_rate, bert_path, classifier_path, dataset, version):
    # Cargar modelos
    bert_model, classifier_model = load_models(bert_path, classifier_path, dataset, version)
    
    # Cargar configuración del dataset
    dataset_config = load_dataset_stats(dataset, version)
    
    # Preprocesar datos
    n_samples = seconds * sample_rate
    data = raw_data[:n_samples]  # Tomar solo los primeros N segundos
    preprocessed_data = preprocess_data(data, window_size=dataset_config.seq_len, stride=dataset_config.seq_len//2)
    
    # Realizar inferencia
    predictions = inference(bert_model, classifier_model, preprocessed_data)
    
    # Agregar predicciones
    final_prediction = np.mean(predictions, axis=0)
    
    # Obtener la categoría con la probabilidad más alta
    category_index = np.argmax(final_prediction)
    category = dataset_config.activity_label[category_index]
    
    return category, final_prediction

# 5. Función para imprimir resultados detallados
def print_detailed_results(category, probabilities, dataset_config):
    print(f"Categoría predicha: {category}")
    print("\nProbabilidades por categoría:")
    for label, prob in zip(dataset_config.activity_label, probabilities):
        print(f"{label}: {prob:.4f}")



In [ ]:
# Ejemplo de uso modificado
BERT_PATH = 'saved/pretrain_base_motion_20_120/motion.pt'
CLASSIFIER_PATH = 'saved/classifier_base_gru_motion_20_120/motion.pt'
DATASET = 'hhar'
VERSION = '20_120'

# Simular datos de IMU (reemplaza esto con tus datos reales)
raw_imu_data = np.random.randn(120, 6)  # 6 seg de datos a 20Hz

# Clasificar 10 segundos de datos
category, probabilities = classify_imu_data(raw_imu_data, seconds=10, sample_rate=100, 
                                            bert_path=BERT_PATH, classifier_path=CLASSIFIER_PATH,
                                            dataset=DATASET, version=VERSION)

# Cargar la configuración del dataset para obtener las etiquetas
dataset_config = load_dataset_stats(DATASET, VERSION)

# Imprimir resultados detallados
print_detailed_results(category, probabilities, dataset_config)

## desacoplando
---------------------

In [ ]:
import torch
import numpy as np
from models import LIMUBertModel4Pretrain, ClassifierGRU
from config import PretrainModelConfig, ClassifierModelConfig, load_model_config, load_dataset_stats


# 1. Configuración y carga de modelos
def load_models(bert_path, classifier_path, dataset, version):
    # Cargar configuraciones
    bert_cfg = load_model_config('pretrain', 'base', 'v1', path_bert='config/limu_bert.json')
    classifier_cfg = load_model_config('classifier', 'gru', 'v1', path_classifier='config/classifier.json')
    
    # Imprimir la configuración del clasificador para verificar
    print("Classifier config:", classifier_cfg)
    
    # Inicializar modelos
    bert_model = LIMUBertModel4Pretrain(bert_cfg, output_embed=True)
    # Asumiendo que el modelo guardado tiene una capa lineal final de 6 a 10
    output_dim = 6  # o el valor correcto basado en tu modelo guardado
    classifier_model = ClassifierGRU(classifier_cfg, input=bert_cfg.hidden, output=output_dim)
    #classifier_model = ClassifierGRU(classifier_cfg, input=bert_cfg.hidden, output=classifier_cfg.linear_io[-1][1])
    
    # Imprimir la estructura del modelo para verificar
    print(classifier_model)
    
    # Cargar pesos pre-entrenados
    bert_model.load_state_dict(torch.load(bert_path, map_location='cpu'))
    
    # Cargar el estado del clasificador
    classifier_state = torch.load(classifier_path, map_location='cpu')
    
    # Imprimir las claves y formas del estado del clasificador
    for key, value in classifier_state.items():
        print(f"{key}: {value.shape}")
    
    # Intenta cargar el estado del clasificador
    classifier_model.load_state_dict(classifier_state, strict=False)
    
    return bert_model, classifier_model

# 2. Preprocesamiento de datos
def preprocess_data(raw_data, window_size=120, stride=60):
    windows = []
    for i in range(0, len(raw_data) - window_size + 1, stride):
        windows.append(raw_data[i:i+window_size])
    return np.array(windows)

# 3. Inferencia (modificada para aceptar modelos precargados)
def inference(bert_model, classifier_model, input_data):
    bert_model.eval()
    classifier_model.eval()
    
    with torch.no_grad():
        embeddings = bert_model(torch.FloatTensor(input_data))
        outputs = classifier_model(embeddings)
        
    return torch.softmax(outputs, dim=1).numpy()

# 4. Función de clasificación (modificada para aceptar modelos precargados)
def classify_imu_data(raw_data, seconds, sample_rate, bert_model, classifier_model, dataset_config):
    # Preprocesar datos
    n_samples = seconds * sample_rate
    data = raw_data[:n_samples]  # Tomar solo los primeros N segundos
    preprocessed_data = preprocess_data(data, window_size=dataset_config.seq_len, stride=dataset_config.seq_len//2)
    
    # Realizar inferencia
    predictions = inference(bert_model, classifier_model, preprocessed_data)
    
    # Agregar predicciones
    final_prediction = np.mean(predictions, axis=0)
    
    # Obtener la categoría con la probabilidad más alta
    category_index = np.argmax(final_prediction)
    category = dataset_config.activity_label[category_index]
    
    return category, final_prediction


# 5. Función para imprimir resultados detallados
def print_detailed_results(category, probabilities, dataset_config):
    print(f"Categoría predicha: {category}")
    print("\nProbabilidades por categoría:")
    for label, prob in zip(dataset_config.activity_label, probabilities):
        print(f"{label}: {prob:.4f}")


# 6. Nueva función para cargar modelos y configuración
def load_models_and_config(bert_path, classifier_path, dataset, version):
    bert_model, classifier_model = load_models(bert_path, classifier_path, dataset, version)
    dataset_config = load_dataset_stats(dataset, version)
    return bert_model, classifier_model, dataset_config

# Función para realizar múltiples inferencias
def run_multiple_inferences(num_inferences):
    for i in range(num_inferences):
        # Simular datos de IMU (reemplaza esto con tus datos reales)
        raw_imu_data = np.random.randn(6000, 6)  # 1 minuto de datos a 100Hz

        # Clasificar 10 segundos de datos
        category, probabilities = classify_imu_data(raw_imu_data, seconds=10, sample_rate=100, 
                                                    bert_model=bert_model, classifier_model=classifier_model,
                                                    dataset_config=dataset_config)

        print(f"\nInferencia {i+1}:")
        print_detailed_results(category, probabilities, dataset_config)


In [ ]:

# Ejemplo de uso modificado
BERT_PATH = 'saved/pretrain_base_motion_20_120/motion.pt'
CLASSIFIER_PATH = 'saved/classifier_base_gru_motion_20_120/motion.pt'
DATASET = 'hhar'
VERSION = '20_120'

# Cargar modelos y configuración una sola vez
bert_model, classifier_model, dataset_config = load_models_and_config(BERT_PATH, CLASSIFIER_PATH, DATASET, VERSION)

In [8]:
raw_imu_data = np.random.randn(120, 6)  # 6 seg de datos a 20Hz


In [18]:
category, probabilities = classify_imu_data(raw_imu_data, seconds=6, sample_rate=20,bert_model=bert_model, classifier_model=classifier_model,dataset_config=dataset_config)
print_detailed_results(category, probabilities, dataset_config)


Categoría predicha: bike

Probabilidades por categoría:
bike: 0.5448
sit: 0.1550
downstairs: 0.0097
upstairs: 0.0650
stand: 0.0950
walk: 0.1306


In [7]:
run_multiple_inferences(3)  # Cambia el número según cuántas inferencias quieras realizar


Inferencia 1:
Categoría predicha: bike

Probabilidades por categoría:
bike: 0.4348
sit: 0.1414
downstairs: 0.0201
upstairs: 0.0569
stand: 0.0901
walk: 0.2567

Inferencia 2:
Categoría predicha: bike

Probabilidades por categoría:
bike: 0.4334
sit: 0.1562
downstairs: 0.0246
upstairs: 0.0897
stand: 0.0987
walk: 0.1973

Inferencia 3:
Categoría predicha: bike

Probabilidades por categoría:
bike: 0.5128
sit: 0.1500
downstairs: 0.0133
upstairs: 0.0634
stand: 0.1001
walk: 0.1604


---------------------
# En duda de uso

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json

# Funciones proporcionadas
def load_config(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    return config

def load_data_and_labels(dataset_name):
    cwd = os.getcwd()
    config_path = os.path.join(cwd, 'dataset/data_config.json')
    config = load_config(config_path)

    data_info = config[dataset_name]
    sr = data_info['sr']
    seq_len = data_info['seq_len']
    dimension = data_info['dimension']
    activity_label_index = data_info['activity_label_index']
    activity_label = data_info['activity_label']
    user_label_index = data_info.get('user_label_index')
    user_label = data_info.get('user_label')
    
    data_file = f'data_{sr}_{seq_len}.npy'
    label_file = f'label_{sr}_{seq_len}.npy'
    
    dataset_folder = os.path.join(cwd,"dataset/",dataset_name.split('_')[0])
    data_path = os.path.join(cwd, dataset_folder, data_file)
    label_path = os.path.join(cwd, dataset_folder, label_file)

    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Data file not found: {data_path}")
    if not os.path.exists(label_path):
        raise FileNotFoundError(f"Label file not found: {label_path}")

    data = np.load(data_path)
    labels = np.load(label_path)
    
    return data, labels, activity_label_index, activity_label, user_label_index, user_label

def create_dataframe(data, labels, activity_label_index, activity_label):
    n_samples, n_windows, n_features = data.shape
    time_index = np.arange(n_samples * n_windows) / (n_windows // 10)  # Calcula los tiempos en segundos
    
    # Aplanar los datos y las etiquetas
    flattened_data = data.reshape(n_samples * n_windows, n_features)
    flattened_labels = labels.reshape(n_samples * n_windows, -1)
    
    # Crear el DataFrame con los datos y las etiquetas
    df = pd.DataFrame(flattened_data, columns=[f'DOF{i+1}' for i in range(n_features)])
    df['Time (s)'] = time_index
    df['Activity'] = [activity_label[int(label)] for label in flattened_labels[:, activity_label_index]]
    
    return df

def plot_data(df, start_second=None, end_second=None, figsize=(12, 8), show_milliseconds=False):
    if start_second is None:
        start_second = df['Time (s)'].min()
    if end_second is None:
        end_second = df['Time (s)'].max()

    df_filtered = df[(df['Time (s)'] >= start_second) & (df['Time (s)'] <= end_second)]

    plt.figure(figsize=figsize)
    
    for i in range(1, df.shape[1] - 1):  # Excluye la columna 'Activity'
        plt.subplot(3, 2, i)
        for activity in df['Activity'].unique():
            activity_data = df_filtered[df_filtered['Activity'] == activity]
            plt.plot(activity_data['Time (s)'], activity_data[f'DOF{i}'], label=activity)
        plt.title(f'DOF{i}')
        plt.xlabel('Time (s)')
        plt.ylabel(f'DOF{i}')
        plt.legend()

        plt.xticks(ticks=plt.xticks()[0], labels=[f'{x:.1f}s' for x in plt.xticks()[0]])
        if show_milliseconds:
            plt.xticks(ticks=plt.xticks()[0], labels=[f'{x:.1f}s ({x*1000:.0f}ms)' for x in plt.xticks()[0]])

    plt.tight_layout()
    plt.show()

def plot_activity(df, activity, start_second=None, end_second=None, figsize=(12, 8)):
    if start_second is None:
        start_second = df['Time (s)'].min()
    if end_second is None:
        end_second = df['Time (s)'].max()

    df_filtered = df[(df['Time (s)'] >= start_second) & (df['Time (s)'] <= end_second) & (df['Activity'] == activity)]

    plt.figure(figsize=figsize)

    # Subgráfico para el acelerómetro (3 DOF)
    plt.subplot(2, 1, 1)
    plt.plot(df_filtered['Time (s)'], df_filtered['DOF1'], label=f'{activity} - DOF1')
    plt.plot(df_filtered['Time (s)'], df_filtered['DOF2'], label=f'{activity} - DOF2')
    plt.plot(df_filtered['Time (s)'], df_filtered['DOF3'], label=f'{activity} - DOF3')
    plt.title(f'{activity} - Acelerómetro (3 DOF)')
    plt.xlabel('Time (s)')
    plt.ylabel('Acceleration')
    plt.legend()
    plt.xlim(start_second, end_second)

    # Subgráfico para el giroscopio (3 DOF)
    plt.subplot(2, 1, 2)
    plt.plot(df_filtered['Time (s)'], df_filtered['DOF4'], label=f'{activity} - DOF4')
    plt.plot(df_filtered['Time (s)'], df_filtered['DOF5'], label=f'{activity} - DOF5')
    plt.plot(df_filtered['Time (s)'], df_filtered['DOF6'], label=f'{activity} - DOF6')
    plt.title(f'{activity} - Giroscopio (3 DOF)')
    plt.xlabel('Time (s)')
    plt.ylabel('Angular Velocity')
    plt.legend()
    plt.xlim(start_second, end_second)

    plt.tight_layout()
    plt.show()

# Script principal
DATASET_NAME = 'hhar_20_120'  # Ajusta esto según tu dataset

# Cargar datos y labels
data, labels, activity_label_index, activity_label, user_label_index, user_label = load_data_and_labels(DATASET_NAME)

print(f"Forma de los datos: {data.shape}")
print(f"Forma de las etiquetas: {labels.shape}")
print(f"Etiquetas de actividad: {activity_label}")

# Crear DataFrame
df = create_dataframe(data, labels, activity_label_index, activity_label)


In [ ]:

# Visualizar una muestra de los datos
plot_data(df, start_second=0, end_second=60)  # Visualiza el primer minuto de datos

# Visualizar una actividad específica
#plot_activity(df, activity=activity_label[0], start_second=0, end_second=60)

# Estadísticas de las labels
print("\nDistribución de labels:")
print(df['Activity'].value_counts(normalize=True))

# Verificar la forma de los datos para el modelo
print(f"\nForma de los datos para el modelo: {data.shape}")
print(f"Cada ventana tiene {data.shape[1]} muestras de tiempo y {data.shape[2]} canales.")
print(f"Duración de cada ventana: {data.shape[1]/20:.2f} segundos")  # Asumiendo sr=20

category, probabilities = classify_imu_data(raw_imu_data, seconds=6, sample_rate=20,bert_model=bert_model, classifier_model=classifier_model,dataset_config=dataset_config)
print_detailed_results(category, probabilities, dataset_config)

In [26]:
def prepare_data_for_inference(df, start_second=None, end_second=None, sample_rate=20, window_size=120):
    # Filtrar el DataFrame por tiempo si se especifica
    if start_second is None:
        start_second = df['Time (s)'].min()
    if end_second is None:
        end_second = df['Time (s)'].max()
    
    df_filtered = df[(df['Time (s)'] >= start_second) & (df['Time (s)'] <= end_second)]
    
    # Seleccionar solo las columnas de datos (DOF1 a DOF6)
    data_columns = [f'DOF{i}' for i in range(1, 7)]
    raw_imu_data = df_filtered[data_columns].values
    
    # Asegurarse de que tenemos suficientes datos para al menos una ventana
    if len(raw_imu_data) < window_size:
        raise ValueError(f"No hay suficientes datos. Se necesitan al menos {window_size} muestras.")
    
    # Recortar los datos para que sean múltiplos exactos del tamaño de la ventana
    num_windows = len(raw_imu_data) // window_size
    raw_imu_data = raw_imu_data[:num_windows * window_size]
    
    # Reshape los datos en ventanas
    raw_imu_data = raw_imu_data.reshape(-1, window_size, 6)
    
    return raw_imu_data

# Ejemplo de uso:
def infer_from_dataframe(df, start_second, end_second, bert_model, classifier_model, dataset_config):
    raw_imu_data = prepare_data_for_inference(df, start_second, end_second)
    
    categories = []
    probabilities = []
    
    for window in raw_imu_data:
        category, probability = classify_imu_data(
            window, 
            seconds=6, 
            sample_rate=20, 
            bert_model=bert_model, 
            classifier_model=classifier_model,
            dataset_config=dataset_config
        )
        categories.append(category)
        probabilities.append(probability)
    
    return categories, probabilities



In [28]:
# Uso
start_second = 0
end_second = 120  # Por ejemplo, inferir sobre el primer minuto de datos
inicio = 2800
tiempo = 120
fin=inicio+ tiempo

categories, probabilities = infer_from_dataframe(df, inicio, fin, bert_model, classifier_model, dataset_config)

print("Categorías inferidas:", categories)
print("Probabilidades:", probabilities)

Categorías inferidas: ['bike', 'bike', 'bike', 'bike', 'bike', 'bike', 'bike', 'bike', 'bike', 'bike', 'bike', 'bike']
Probabilidades: [array([0.52050996, 0.21361014, 0.00652829, 0.07260227, 0.0889217 ,
       0.09782758], dtype=float32), array([0.51982397, 0.21413289, 0.00653117, 0.0727672 , 0.0887173 ,
       0.09802746], dtype=float32), array([0.5206118 , 0.2135221 , 0.00652828, 0.0726058 , 0.08895447,
       0.09777745], dtype=float32), array([0.520695  , 0.21345215, 0.00652845, 0.07257793, 0.08897798,
       0.09776858], dtype=float32), array([0.5198233 , 0.21420579, 0.00652678, 0.07277516, 0.08873717,
       0.09793176], dtype=float32), array([0.520083  , 0.21396807, 0.00652784, 0.07274228, 0.0888093 ,
       0.0978694 ], dtype=float32), array([0.5206189 , 0.21352921, 0.00652753, 0.07259905, 0.08895869,
       0.09776655], dtype=float32), array([0.5208229 , 0.21339713, 0.00652525, 0.07256636, 0.08903143,
       0.09765697], dtype=float32), array([0.5209302 , 0.21330954, 0.0065250